Несмотря на автоматизацию области, очень многие решения в процессе решения задачи принимает data scientist. Некоторые из них автоматизировать достаточно сложно — такие, как выбор правильной метрики, однако есть и такие, которые можно принимать автоматизированно. Многие алгоритмы, как нейронные сети, так и классические алгоритмы обучения, оперируют понятием <<гиперпараметры>> — параметры, которые не учатся в процессе анализа данных, а которые необходимо задать перед началом обучения инженеру самостоятельно. К примеру, в решающих деревьях, такими параметрами являются максимальная глубина дерева и минимальное количество экзмепляров данных в листе. В процессе построения нейросети также есть множество гиперпараметров, которые устанавливает инженер и об их автоматическом выборе мы сегодня поговорим.

Как мы уже сказали, гиперпараметры — это параметры, которые не учатся автоматически, а задаются самостоятельно. В приложении к нейронным сетям такими гиперпараметрами будут:
* Архитектура сети, т.е. выбранные слои и их связность
* Гиперпараметры каждого слоя отдельно, такие как количество нейронов, функции активации, и т.д.
* Параметры оптимизатора — коэффициент скорости обучения и т.д.
* Параметры обучения
* И т.д.

Некоторые из этих гиперпараметров выбираются инженером из его опыта или общепринятых практик — например, архитектура сети. Однако, такие гиперпараметры, как количество нейронов в слое или функция активации, сложно угадать — в разных задачах наиболее оптимальным выбором будут разные варианты и их выбор неочевиден. Данные параметры можно попробовать подобрать автоматически — натренировать несколько моделей с разными комбинациями параметров и выбрать из них лучшую.  

Есть несколько подходов к подбору гиперпараметров. Классически используемый подход — это т.н. GridSearch. GridSearch — это подход, в котором для каждого параметра указывается весь список значений, который нужно проверить. При N параметров получается N-мерное поле, все точки которого необходимо посетить и посчитать качество модели в этой точке, после чего выбрать наиболее оптимальную. Главным недостатком данного подхода является огромная вычислительная сложность - алгоритм будет честно проверять все возможные комбинации параметров и искать среди них оптимальную.

Альтернативным подходом является байесовская оптимизация. Представим некую функцию $G(x_1, x_2, ..., x_i)$, где $x_i$ — это гиперпараметр, а результат этой функции — качество модели. Байесовская оптимизация опирается на предположение, что данная функция является непрерывной, т.е. малые изменения аргументов приведут к малым изменениям значения функции. Учитывая это предположение, алгоритмы байесовской оптимизации стараются найти области наиболее высоких значений параметров, а потом найти оптимальную точку уже в заданной области.

В рамках данной лекции мы с вами возьмем один из самых популярных пакетов для байесовской оптимизации — Optuna — и используем его для поиска гиперпараметров для нашей сети.

Давайте установим пакет и проверим, что фреймворк TensorFlow и пакет TensorFlow_Dataset тоже установлены.

In [1]:
!pip install optuna
!pip install numpy tensorflow tensorflow_datasets

     |████████████████████████████████| 308 kB 4.7 MB/s 
     |████████████████████████████████| 80 kB 4.9 MB/s 
     |████████████████████████████████| 209 kB 42.8 MB/s 
     |████████████████████████████████| 75 kB 2.7 MB/s 
     |████████████████████████████████| 112 kB 36.6 MB/s 
     |████████████████████████████████| 144 kB 47.3 MB/s 
     |████████████████████████████████| 49 kB 3.6 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=faae74d9874701419da84746bf761cb599a3680762cbdf1f1ec9fd6b4d8458a4
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


Тестировать пакет Optuna мы будем на том же самом примере, что и ранее — на наборе данных "Ирисы Фишера" и нашей маленькой нейронной сети из трех слоев. Давайте импортируем все необходимые пакеты, в т.ч. Optuna.

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import metrics, layers, activations, optimizers, losses
import tensorflow_datasets as tfds

import optuna

Как и ранее, разделим набор данных на две части, одна из которых будет использоваться для обучения, а вторая - для валидации модели, и укажем дополнительные параметры. 

In [3]:
ds_train, ds_test = tfds.load(
    name='iris',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True
)

input_shape = (4, )  
batch_size = 10      
amount_of_classes = 3

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/iris/2.0.0.incompleteN0L3X2/iris-train.tfrecord


  0%|          | 0/150 [00:00<?, ? examples/s]

Dataset iris downloaded and prepared to /root/tensorflow_datasets/iris/2.0.0. Subsequent calls will reuse this data.


Аналогичным образом преобразуем набор данных с помощью one-hot encoding, перемешаем и разделим на кусочки одинакового размера.

In [4]:
def make_one_hot(x, y):
    return x, tf.one_hot(y, depth=amount_of_classes)

ds_train = (
    ds_train
    .map(make_one_hot)
    .shuffle(len(ds_train))
    .batch(batch_size, drop_remainder=True)
)
    
ds_test = (
    ds_test
    .map(make_one_hot)
    .batch(batch_size, drop_remainder=True)
)

Далее создадим модель, чтобы посмотреть на качество при сдандартных значениях. Используем ту же самую модель, что и в предыдущем примере и обучим на протяжении 10 эпох.

In [5]:
model = keras.Sequential()
model.add(layers.Dense(32, input_shape=input_shape, activation='sigmoid'))
model.add(layers.Dense(16, activation='sigmoid'))
model.add(layers.Dense(amount_of_classes, activation=activations.softmax))

model.compile(
    optimizer=optimizers.Adam(learning_rate=0.003),
    loss=losses.CategoricalCrossentropy(),
    metrics=[metrics.CategoricalAccuracy()]
)

history = model.fit(ds_train, epochs=10, validation_data=ds_test, verbose=1)

Epoch 1/10
12/12 [==============================] - 1s 24ms/step - loss: 1.2824 - categorical_accuracy: 0.3333 - val_loss: 1.1204 - val_categorical_accuracy: 0.3333
Epoch 2/10
12/12 [==============================] - 0s 9ms/step - loss: 1.1305 - categorical_accuracy: 0.3000 - val_loss: 1.0887 - val_categorical_accuracy: 0.3000
Epoch 3/10
12/12 [==============================] - 0s 9ms/step - loss: 1.0735 - categorical_accuracy: 0.3667 - val_loss: 1.0549 - val_categorical_accuracy: 0.5333
Epoch 4/10
12/12 [==============================] - 0s 10ms/step - loss: 1.0315 - categorical_accuracy: 0.6667 - val_loss: 1.0098 - val_categorical_accuracy: 0.6333
Epoch 5/10
12/12 [==============================] - 0s 9ms/step - loss: 0.9918 - categorical_accuracy: 0.6750 - val_loss: 0.9688 - val_categorical_accuracy: 0.6333
Epoch 6/10
12/12 [==============================] - 0s 10ms/step - loss: 0.9426 - categorical_accuracy: 0.6750 - val_loss: 0.9190 - val_categorical_accuracy: 0.6333
Epoch 7/10
12

Отметьте, какое значение точности на валидационном наборе данных мы получили. Вследствие того, что веса инициируются случайным образом, процесс обучения достаточно стохастичен и случайно можно получить удачный набор весов, который приведет к очень высокой точности с первого раза. Однако, чаще всего точность после подбора гиперпараметров оказывается выше вследствие нахождения более удачного сочетания. Сравните в конце и отметьте, что получилось в вашем случае.

Для того, чтобы оптимизировать модель, фреймворку Optuna необходимо создать цель оптимизации - функцию, которую фреймворк будет запускать. Выходом функции должно быть одно число, которое необходимо максимизировать или минимизировать.

Функция должна быть полностью самостоятельной, все необходимые инициализации следует совершать в ней. Стоит также учитывать, что фреймворк Optuna можно использовать с использованием внешней базы данных и запуская множество процессов, которые будут совершать оптимизацию параллельно, поэтому не стоит использовать глобальные переменные, если они не являются константами.

Функция принимает на вход единственный параметр - объект optuna.trial.Trial, который в дальнейшем в ней используется. Давайте инициализируем новую модель нейронной сети в данной функции и дадим фреймворку Optuna оптимизировать количество нейронов в слоях и используемые функции активации. Для этого вместо указания числа нейронов и функции активации указывается то, как этот параметр можно выбирать - с помощью объекта trial. Optuna позволяет выбирать из нескольких видов распределений и разных типов данных, а также указывать категориальное равномерное распределение с заранее заданными объектами, если вам нужно указать несколько доступных вариантов.

Для количество нейронов первого слоя укажем возможность выбора любого целого числа в промежутке от 16 до 64х, а второго слоя - от 4 до 16 нейронов. Функции активации в обоих случаях укажем как одну из двух - либо сигмоиду, либо ReLU.

Также зададим коэффициент скорости обучения как гиперпараметр и укажем выбор его из равномерного распределения с границами 0.001 и 0.01.

Далее в рамках этой же функции обучим модель и финальным результатом обучения укажем последнюю точность на валидационном наборе данных, которая записана в объекте history.

In [ ]:
def optuna_objective(trial: optuna.trial.Trial):
    model = keras.Sequential(
        [
            layers.Dense(
                units=trial.suggest_int('l1_neurons', 16, 64),
                input_shape=input_shape,
                activation=trial.suggest_categorical('l1_activation', ['sigmoid', 'relu'])
            ),
            layers.Dense(
                units=trial.suggest_int('l2_neurons', 4, 16),
                activation=trial.suggest_categorical('l2_activation', ['sigmoid', 'relu'])
            ),
            layers.Dense(amount_of_classes, activation=activations.softmax)
        ]
    )
    
    model.compile(
        optimizer=optimizers.Adam(learning_rate=trial.suggest_uniform('lr', 0.001, 0.01)),
        loss=losses.CategoricalCrossentropy(),
        metrics=[metrics.CategoricalAccuracy()]
    )
    
    history = model.fit(ds_train, epochs=10, validation_data=ds_test, verbose=0)
    accuracy = history.history['val_categorical_accuracy'][-1]
    return accuracy

После определения функции оптимизации необходимо создать исследование - т.е. инициализировать специальный объект. В данном случае мы укажем, что хотим максимизировать выход функции.

In [ ]:
study = optuna.create_study(direction='maximize')

[I 2021-08-25 14:03:52,433] A new study created in memory with name: no-name-27bde880-f456-4d5b-9bac-abf12bfceab5


И далее запустим процесс оптимизации, указав функцию для оптимизации и количество попыток.

In [ ]:
study.optimize(optuna_objective, n_trials=30)

[I 2021-08-25 14:03:54,865] Trial 0 finished with value: 0.8999999761581421 and parameters: {'l1_neurons': 34, 'l1_activation': 'sigmoid', 'l2_neurons': 15, 'l2_activation': 'relu', 'lr': 0.009611589181367737}. Best is trial 0 with value: 0.8999999761581421.
[I 2021-08-25 14:03:57,386] Trial 1 finished with value: 0.8999999761581421 and parameters: {'l1_neurons': 59, 'l1_activation': 'relu', 'l2_neurons': 16, 'l2_activation': 'relu', 'lr': 0.00499632814924884}. Best is trial 0 with value: 0.8999999761581421.
[I 2021-08-25 14:03:59,578] Trial 2 finished with value: 0.9666666388511658 and parameters: {'l1_neurons': 18, 'l1_activation': 'sigmoid', 'l2_neurons': 7, 'l2_activation': 'sigmoid', 'lr': 0.00882599249157668}. Best is trial 2 with value: 0.9666666388511658.
[I 2021-08-25 14:04:01,859] Trial 3 finished with value: 0.3333333432674408 and parameters: {'l1_neurons': 42, 'l1_activation': 'sigmoid', 'l2_neurons': 9, 'l2_activation': 'relu', 'lr': 0.0031060334614656992}. Best is trial 2

После этого в объекте study будет храниться информация о лучшей попытке и использованных параметрах. Мы можем выбрать эти параметры как оптимальные и использовать в дальнейшем для нашей модели.

In [ ]:
study.best_params

{'l1_neurons': 18,
 'l1_activation': 'sigmoid',
 'l2_neurons': 7,
 'l2_activation': 'sigmoid',
 'lr': 0.00882599249157668}

Фреймворк Optuna обладает достаточно большими возможностями, не рассмотренными в данной лекции, которые могут вам пригодиться. Возможно, самый нужный из них - это возможность использования внешнего хранилища данных, такого как база данных MySQL или SQLite, что позволит вам запустить несколько процессов оптимизации параллельно, а также останавливать и продолжать процесс поиска гиперпараметров.